# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
import numpy as np
import pandas as pd
import datapackage as dp
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [34]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
497,YUM
498,ZBH
499,ZBRA
500,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [61]:
from secrt import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'pk_7939993b9d3a422a8fe06bd3467b843c'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [22]:
symbol='AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price=data['latestPrice']
market_cap=data['marketCap']
print(market_cap/1000000000000)

0.036272316934


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns= [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe=pd.DataFrame( columns = my_columns)


In [24]:
final_dataframe = final_dataframe._append(pd.Series(
        [
        symbol,
        price,
        market_cap,
        'N/A'
    ], index= my_columns

),
    
    ignore_index=True #important when appending data to a dataframe
 )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AME,157.16,36272316934,N/A
1,AAPL,157.16,36272316934,N/A
2,AAPL,157.16,36272316934,N/A


In [8]:
#final_dataframe=final_dataframe._append(new_row, ignore_index=True)
#final_dataframe      #.append changed to  ._append 

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [47]:
final_dataframe=pd.DataFrame(columns= my_columns)
for stock in stocks['Ticker'][:100]:    #
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    print(data)
    final_dataframe= final_dataframe._append(pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
        
    ],
    index=my_columns),
        ignore_index=True
                           )
    


{'avgTotalVolume': 2181549, 'calculationPrice': 'close', 'change': -1.48, 'changePercent': -0.0115, 'close': 127.2, 'closeSource': 'official', 'closeTime': 1702069202296, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': 127.145, 'delayedPriceTime': 1702069180017, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 127.2, 'extendedPriceTime': 1702080000003, 'high': 129, 'highSource': '15 minute delayed price', 'highTime': 1702069199978, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 127.22, 'iexCloseTime': 1702069190502, 'iexLastUpdated': 1702069190502, 'iexMarketPercent': 0.026150949116665764, 'iexOpen': 127.735, 'iexOpenTime': 1702045875113, 'iexRealtimePrice': 127.22, 'iexRealtimeSize': 60, 'iexVolume': 37607, 'lastTradeTime': 1702069199208, 'latestPrice': 127.2, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069202296, 'latestVolume': 1438074, 'low': 126.97, 'lowSource': '1

{'avgTotalVolume': 5909360, 'calculationPrice': 'close', 'change': 0.46, 'changePercent': 0.00442, 'close': 104.51, 'closeSource': 'official', 'closeTime': 1702069202316, 'companyName': 'Abbott Laboratories', 'currency': 'USD', 'delayedPrice': 104.57, 'delayedPriceTime': 1702069161453, 'extendedChange': 0.01, 'extendedChangePercent': 0.0001, 'extendedPrice': 104.52, 'extendedPriceTime': 1702083253409, 'high': 104.86, 'highSource': '15 minute delayed price', 'highTime': 1702069199998, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 104.52, 'iexCloseTime': 1702069193115, 'iexLastUpdated': 1702069193115, 'iexMarketPercent': 0.018302010457624675, 'iexOpen': 104.26, 'iexOpenTime': 1702045869705, 'iexRealtimePrice': 104.52, 'iexRealtimeSize': 54, 'iexVolume': 62717, 'lastTradeTime': 1702069199998, 'latestPrice': 104.51, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069202316, 'latestVolume': 3426782, 'low': 104.12, 'lowSource

{'avgTotalVolume': 1649501, 'calculationPrice': 'close', 'change': 0.17, 'changePercent': 0.00076, 'close': 224.01, 'closeSource': 'official', 'closeTime': 1702069200026, 'companyName': 'Autodesk Inc.', 'currency': 'USD', 'delayedPrice': 224.09, 'delayedPriceTime': 1702069178485, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 224.01, 'extendedPriceTime': 1702083534199, 'high': 225.31, 'highSource': '15 minute delayed price', 'highTime': 1702069199938, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 224.01, 'iexCloseTime': 1702069199360, 'iexLastUpdated': 1702069199360, 'iexMarketPercent': 0.0306063908058823, 'iexOpen': 222.385, 'iexOpenTime': 1702045802661, 'iexRealtimePrice': 224.01, 'iexRealtimeSize': 100, 'iexVolume': 32863, 'lastTradeTime': 1702069199905, 'latestPrice': 224.01, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069200026, 'latestVolume': 1073730, 'low': 221.895, 'lowSource': '15 minute

{'avgTotalVolume': 944042, 'calculationPrice': 'close', 'change': 0.01, 'changePercent': 0.0014, 'close': 7.14, 'closeSource': 'official', 'closeTime': 1702069201975, 'companyName': 'Apartment Investment & Management Co. - Class A', 'currency': 'USD', 'delayedPrice': 7.15, 'delayedPriceTime': 1702069180128, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 7.14, 'extendedPriceTime': 1702080000002, 'high': 7.175, 'highSource': '15 minute delayed price', 'highTime': 1702069199978, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 7.16, 'iexCloseTime': 1702069196478, 'iexLastUpdated': 1702069196478, 'iexMarketPercent': 0.06051700380640035, 'iexOpen': 7.13, 'iexOpenTime': 1702045996041, 'iexRealtimePrice': 7.16, 'iexRealtimeSize': 3, 'iexVolume': 40669, 'lastTradeTime': 1702069197312, 'latestPrice': 7.14, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069201975, 'latestVolume': 672026, 'low': 7.05, 'lowSource':

{'avgTotalVolume': 3887949, 'calculationPrice': 'close', 'change': -0.36, 'changePercent': -0.01001, 'close': 35.61, 'closeSource': 'official', 'closeTime': 1702069201963, 'companyName': 'Alaska Air Group Inc.', 'currency': 'USD', 'delayedPrice': 35.62, 'delayedPriceTime': 1702069183829, 'extendedChange': 0.11, 'extendedChangePercent': 0.00309, 'extendedPrice': 35.72, 'extendedPriceTime': 1702083493150, 'high': 36.08, 'highSource': '15 minute delayed price', 'highTime': 1702069199992, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 35.56, 'iexCloseTime': 1702069198019, 'iexLastUpdated': 1702069198019, 'iexMarketPercent': 0.05170123731909023, 'iexOpen': 35.92, 'iexOpenTime': 1702045800875, 'iexRealtimePrice': 35.56, 'iexRealtimeSize': 78, 'iexVolume': 219847, 'lastTradeTime': 1702069199988, 'latestPrice': 35.61, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069201963, 'latestVolume': 4252258, 'low': 35.415, 'lowSource': 

...

{'avgTotalVolume': 16718596, 'calculationPrice': 'close', 'change': 0.39, 'changePercent': 0.00804, 'close': 48.89, 'closeSource': 'official', 'closeTime': 1702069201998, 'companyName': 'Citigroup Inc', 'currency': 'USD', 'delayedPrice': 48.885, 'delayedPriceTime': 1702069187770, 'extendedChange': 0.12, 'extendedChangePercent': 0.00245, 'extendedPrice': 49.01, 'extendedPriceTime': 1702083571190, 'high': 49.17, 'highSource': '15 minute delayed price', 'highTime': 1702069199993, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 48.88, 'iexCloseTime': 1702069195940, 'iexLastUpdated': 1702069195940, 'iexMarketPercent': 0.022602931835289962, 'iexOpen': 48.435, 'iexOpenTime': 1702045860306, 'iexRealtimePrice': 48.88, 'iexRealtimeSize': 100, 'iexVolume': 397615, 'lastTradeTime': 1702069199978, 'latestPrice': 48.89, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069201998, 'latestVolume': 17591302, 'low': 48.42, 'lowSource': '15 m

{'avgTotalVolume': 916655, 'calculationPrice': 'close', 'change': 1.3, 'changePercent': 0.00733, 'close': 178.56, 'closeSource': 'official', 'closeTime': 1702069200347, 'companyName': 'Cboe Global Markets Inc.', 'currency': 'USD', 'delayedPrice': 178.59, 'delayedPriceTime': 1702069195005, 'extendedChange': 0.16, 'extendedChangePercent': 0.0009, 'extendedPrice': 178.72, 'extendedPriceTime': 1702083600854, 'high': 179.08, 'highSource': '15 minute delayed price', 'highTime': 1702069200003, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 178.55, 'iexCloseTime': 1702069197696, 'iexLastUpdated': 1702069197696, 'iexMarketPercent': 0.07820947664335179, 'iexOpen': 176.68, 'iexOpenTime': 1702045801311, 'iexRealtimePrice': 178.55, 'iexRealtimeSize': 9, 'iexVolume': 63042, 'lastTradeTime': 1702069197696, 'latestPrice': 178.56, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069200347, 'latestVolume': 806066, 'low': 176.01, 'lowSource

{'avgTotalVolume': 925511, 'calculationPrice': 'close', 'change': -0.36, 'changePercent': -0.00254, 'close': 141.35, 'closeSource': 'official', 'closeTime': 1702069201995, 'companyName': 'Celanese Corp - Series A', 'currency': 'USD', 'delayedPrice': 141.37, 'delayedPriceTime': 1702069183634, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 141.35, 'extendedPriceTime': 1702080000001, 'high': 143.47, 'highSource': '15 minute delayed price', 'highTime': 1702069199979, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 141.33, 'iexCloseTime': 1702069199143, 'iexLastUpdated': 1702069199143, 'iexMarketPercent': 0.09778650634778487, 'iexOpen': 141.715, 'iexOpenTime': 1702045834620, 'iexRealtimePrice': 141.33, 'iexRealtimeSize': 10, 'iexVolume': 59401, 'lastTradeTime': 1702069199955, 'latestPrice': 141.35, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069201995, 'latestVolume': 607456, 'low': 140.91, 'lowSource': 

{'avgTotalVolume': 1013811, 'calculationPrice': 'close', 'change': -1.11, 'changePercent': -0.00301, 'close': 367.55, 'closeSource': 'official', 'closeTime': 1702069200266, 'companyName': 'Charter Communications Inc. - Class A', 'currency': 'USD', 'delayedPrice': 367.41, 'delayedPriceTime': 1702069199905, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 367.55, 'extendedPriceTime': 1702072906127, 'high': 371.15, 'highSource': 'IEX real time price', 'highTime': 1702049120871, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 367.41, 'iexCloseTime': 1702069197602, 'iexLastUpdated': 1702069197602, 'iexMarketPercent': 0.05106775954275632, 'iexOpen': 365.57, 'iexOpenTime': 1702045902857, 'iexRealtimePrice': 367.41, 'iexRealtimeSize': 97, 'iexVolume': 44460, 'lastTradeTime': 1702069199905, 'latestPrice': 367.55, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069200266, 'latestVolume': 870608, 'low': 363.47, 'low

{'avgTotalVolume': 17605046, 'calculationPrice': 'close', 'change': -0.07, 'changePercent': -0.00166, 'close': 42.1, 'closeSource': 'official', 'closeTime': 1702069200173, 'companyName': 'Comcast Corp - Class A', 'currency': 'USD', 'delayedPrice': 42.09, 'delayedPriceTime': 1702069168540, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 42.1, 'extendedPriceTime': 1702082082443, 'high': 42.24, 'highSource': '15 minute delayed price', 'highTime': 1702069200000, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 42.08, 'iexCloseTime': 1702069199054, 'iexLastUpdated': 1702069199054, 'iexMarketPercent': 0.024008670359612343, 'iexOpen': 41.72, 'iexOpenTime': 1702045800915, 'iexRealtimePrice': 42.08, 'iexRealtimeSize': 2, 'iexVolume': 482989, 'lastTradeTime': 1702069199079, 'latestPrice': 42.1, 'latestSource': 'Close', 'latestTime': 'December 8, 2023', 'latestUpdate': 1702069200173, 'latestVolume': 20117274, 'low': 41.66, 'lowSource': '15 minute

In [48]:
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.20,37217101126,N/A
1,AAL,13.76,8992717968,N/A
2,AAP,56.27,3348176977,N/A
3,AAPL,195.71,3043829093920,N/A
4,ABBV,149.28,263559426207,N/A
...,...,...,...,...
95,CMA,49.97,6589684416,N/A
96,CMCSA,42.10,169058249245,N/A
97,CME,211.62,76181049518,N/A
98,CMG,2242.95,61557000147,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [59]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [73]:
symbol_groups=list(chunks(stocks['Ticker'][:100],25))
symbol_strings=[]
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe=pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:    
    batch_api_call_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe._append(pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ], index= my_columns
        ),ignore_index=True
                                               )
        
final_dataframe
        

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.20,37217101126,N/A
1,AAL,13.76,8992717968,N/A
2,AAP,56.27,3348176977,N/A
3,AAPL,195.71,3043829093920,N/A
4,ABBV,149.28,263559426207,N/A
...,...,...,...,...
95,CMA,49.97,6589684416,N/A
96,CMCSA,42.10,169058249245,N/A
97,CME,211.62,76181049518,N/A
98,CMG,2242.95,61557000147,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [86]:
portfolio_size=input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number!  \nplease try again:")
    portfolio_size=input('Enter the value of your portfolio:')
    val = float(portfolio_size)
    


Enter the value of your portfolio:4560000


In [91]:
position_size= val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
    
final_dataframe    
#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.20,37217101126,358
1,AAL,13.76,8992717968,3313
2,AAP,56.27,3348176977,810
3,AAPL,195.71,3043829093920,232
4,ABBV,149.28,263559426207,305
...,...,...,...,...
95,CMA,49.97,6589684416,912
96,CMCSA,42.10,169058249245,1083
97,CME,211.62,76181049518,215
98,CMG,2242.95,61557000147,20


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [129]:
writer= pd.ExcelWriter('recommended trades.xlsx', engine= 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [130]:
background_color='#0a0a23'
font_color='#ffffff'

string_format= writer.book.add_format(
{
    'font_color': font_color,
    'bg_color':background_color,
    'border':1
}
)

dollar_format= writer.book.add_format(
{
    'num_format':'$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border':1
}
)

integer_format= writer.book.add_format(
{
    'num_format':'0',
    'font_color': font_color,
    'bg_color' : background_color,
    'border':1
}
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [131]:
#writer.sheets['Recommended Trades'].set_column('A:A',18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18, string_format)
#writer._save()
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [132]:
column_formats={
    'A':['Ticker', string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0],string_format)

writer._save()

## Saving Our Excel Output

Saving our Excel file is very easy:

In [134]:
writer._save()